In [2]:
import numpy as np
import cvxpy as cp

# Constants
num_elements = 31  # Number of elements in the transmitter array
wavelength = 1  # Assume λ = 1 for simplicity
d = wavelength / 2  # Inter-element spacing

# Receiver and Scatterer Locations
receiver_angles = np.radians([-30, 0, 30])  # Convert degrees to radians
scatterers = np.array([[40, -30], [70, 50]])  # (x, y) in wavelengths

# Steering Vector Function
def steering_vector(theta, num_elements, d, wavelength):
    k = 2 * np.pi / wavelength
    positions = np.arange(num_elements) * d
    return np.exp(1j * k * positions * np.sin(theta))

# Construct the steering matrix A_hat
A_hat = np.column_stack([steering_vector(theta, num_elements, d, wavelength) for theta in receiver_angles])

# Define Optimization Variables
x1 = cp.Variable(num_elements, complex=True)
x2 = cp.Variable(num_elements, complex=True)
x3 = cp.Variable(num_elements, complex=True)

# Canonical Basis Vectors (3D space)
u1 = np.array([1, 0, 0])
u2 = np.array([0, 1, 0])
u3 = np.array([0, 0, 1])

# Define the objective function
objective = cp.Minimize(
    cp.norm(A_hat.T @ x1 - u1, 2) ** 2 +
    cp.norm(A_hat.T @ x2 - u2, 2) ** 2 +
    cp.norm(A_hat.T @ x3 - u3, 2) ** 2
)

# Define Constraints
m_db = -40  # m in dB
m = 10 ** (m_db / 20)  # Convert dB to linear scale
QPSK_phases = np.array([0, np.pi/2, np.pi, 3*np.pi/2])

constraints = []
for phi in QPSK_phases:
    for psi in QPSK_phases:
        constraints.append(
            cp.norm(A_hat.T @ (x1 + x2 * np.exp(1j * phi) + x3 * np.exp(1j * psi)), 2) ** 2 <= m ** 2
        )

# Solve the optimization problem
prob = cp.Problem(objective, constraints)
prob.solve()

# Print results
print("Optimal x1:", x1.value)
print("Optimal x2:", x2.value)
print("Optimal x3:", x3.value)

x_1 = [ (1.80470398e-04-6.25053735e-06j),  (2.74302947e-07+1.86964196e-04j),
       (-1.80705129e-04-6.34260784e-06j), (-3.54825415e-08-1.98829882e-04j),
  (1.80645588e-04-6.42122076e-06j),  8.64919072e-08+1.86578967e-04j,
 -1.80192288e-04-6.20383442e-06j,  1.07245136e-07-1.98830990e-04j,
  1.80174474e-04-5.94136341e-06j, -2.00558132e-07+1.86698859e-04j,
 -1.80438549e-04-6.26404409e-06j, -1.92774227e-08-1.98981516e-04j,
  1.80317178e-04-6.14829647e-06j, -9.89834796e-08+1.86629645e-04j,
 -1.80569265e-04-6.36583225e-06j,  4.37043927e-09-1.99360823e-04j,
  1.80238672e-04-6.35289874e-06j,  3.11058535e-08+1.86574812e-04j,
 -1.80477124e-04-6.11491022e-06j, -1.48014967e-07-1.99305327e-04j,
  1.80368585e-04-6.19634700e-06j, -2.87522956e-08+1.86754422e-04j,
 -1.80565330e-04-6.22432758e-06j,  3.44733904e-07-1.99133850e-04j,
  1.80540419e-04-6.55535297e-06j,  1.06585074e-07+1.86581773e-04j,
 -1.80126988e-04-6.21080785e-06j, -1.16307778e-07-1.98878943e-04j,
  1.80486541e-04-6.19177138e-06j, -3.28824802e-08+1.86654288e-04j,
 -1.80441753e-04-6.33146611e-06j]
x_2 = [0.00018683-1.93452300e-07j, 0.00017462-1.26518008e-07j,
 0.00018782+1.88247724e-07j, 0.00019804+1.65341920e-07j,
 0.00018699+1.35308987e-07j, 0.00017428+3.42736554e-09j,
 0.00018624-1.85517406e-07j, 0.00019988-1.88810074e-07j,
 0.0001867 +1.29897451e-07j, 0.00017434+1.09332959e-07j,
 0.00018683+1.88707864e-07j, 0.00019883+7.44197867e-08j,
 0.00018671-5.02644763e-08j, 0.00017463-3.05371376e-07j,
 0.0001874 +1.35549690e-07j, 0.00019905-3.51908281e-08j,
 0.00018674+7.78588109e-08j, 0.00017398-1.59816854e-07j,
 0.00018633-2.48274855e-07j, 0.00019934+1.07047667e-07j,
 0.00018657+1.27313500e-07j, 0.00017356+1.03640636e-07j,
 0.00018623-1.03439428e-07j, 0.00019829-3.08211728e-07j,
 0.00018662+5.31828137e-08j, 0.00017363+1.24341953e-07j,
 0.00018644+1.85930683e-07j, 0.00019933-3.56386189e-08j,
 0.00018686-5.88115120e-08j, 0.00017372+2.99301030e-08j,
 0.00018672+5.98378926e-08j]
x_3 = [ 1.80300750e-04+6.06600892e-06j,  1.97712961e-08-1.86701436e-04j,
 -1.80573702e-04+6.13650933e-06j,  1.04851552e-07+1.99196580e-04j,
  1.80499657e-04+6.07068339e-06j,  4.52702905e-08-1.86623836e-04j,
 -1.80672569e-04+6.11513055e-06j,  2.96672286e-08+1.99292968e-04j,
  1.80379984e-04+6.13612175e-06j,  2.22565389e-08-1.86094761e-04j,
 -1.80439198e-04+6.10100763e-06j,  1.24631587e-07+1.99055815e-04j,
  1.79965731e-04+6.16910939e-06j,  1.12280962e-07-1.86325318e-04j,
 -1.80187012e-04+6.03251464e-06j,  1.04369305e-07+1.99118789e-04j,
  1.80479861e-04+6.12657429e-06j,  1.48753320e-07-1.86566008e-04j,
 -1.80486382e-04+6.17766646e-06j,  1.58566126e-07+1.99323084e-04j,
  1.80557660e-04+6.08590495e-06j,  1.41199840e-07-1.86447197e-04j,
 -1.80552493e-04+6.13443871e-06j,  1.22390986e-07+1.99343836e-04j,
  1.80239661e-04+6.11077310e-06j,  6.94657822e-08-1.86617259e-04j,
 -1.80763183e-04+6.07440292e-06j,  6.43923681e-08+1.99164737e-04j,
  1.80246938e-04+6.11814355e-06j,  1.49913734e-07-1.86886669e-04j,
  -1.80413497e-04+6.11169073e-06j]
print(np.array(x_1).dtype)

Optimal x1: [ 1.80470398e-04-6.25053735e-06j  2.74302947e-07+1.86964196e-04j
 -1.80705129e-04-6.34260784e-06j -3.54825415e-08-1.98829882e-04j
  1.80645588e-04-6.42122076e-06j  8.64919072e-08+1.86578967e-04j
 -1.80192288e-04-6.20383442e-06j  1.07245136e-07-1.98830990e-04j
  1.80174474e-04-5.94136341e-06j -2.00558132e-07+1.86698859e-04j
 -1.80438549e-04-6.26404409e-06j -1.92774227e-08-1.98981516e-04j
  1.80317178e-04-6.14829647e-06j -9.89834796e-08+1.86629645e-04j
 -1.80569265e-04-6.36583225e-06j  4.37043927e-09-1.99360823e-04j
  1.80238672e-04-6.35289874e-06j  3.11058535e-08+1.86574812e-04j
 -1.80477124e-04-6.11491022e-06j -1.48014967e-07-1.99305327e-04j
  1.80368585e-04-6.19634700e-06j -2.87522956e-08+1.86754422e-04j
 -1.80565330e-04-6.22432758e-06j  3.44733904e-07-1.99133850e-04j
  1.80540419e-04-6.55535297e-06j  1.06585074e-07+1.86581773e-04j
 -1.80126988e-04-6.21080785e-06j -1.16307778e-07-1.98878943e-04j
  1.80486541e-04-6.19177138e-06j -3.28824802e-08+1.86654288e-04j
 -1.80441753e

In [4]:
import numpy as np
import scipy.linalg as linalg
import matplotlib.pyplot as plt

# Define the number of elements and the wavelength
N = 31  # Number of elements
lambda_ = 1  # Wavelength (normalized)
d = lambda_ / 2  # Inter-element spacing (lambda/2)
k = 2 * np.pi / lambda_  # Wavenumber

# Define the desired directions (angles in degrees) for MIMO system
theta_desired = [30,0,-30]  # Example: Two desired directions for MIMO

# Define interference directions (angles in degrees)
theta_interference = [90]  # Directions where nulls are desired
theta_interference = np.append(theta_interference, np.ones(31 - len(theta_interference) - len(theta_desired))*90)

# Function to compute steering vector for a given angle
def steering_vector(theta, N, d, k,x_opt):
    theta_rad = np.radians(theta)  # Convert to radians
    n = np.arange(N)  # Element indices
    a = x_opt*np.exp(1j * k * d * n * np.sin(theta_rad))  # Steering vector
    return a


x_opt = x1#x1_opt

# Compute the steering vectors for the desired and interference directions
#print("Type of a_d:", type(a_d))
#for idx, ad in enumerate(a_d):
#    print(f"Type of a_d[{idx}]:", type(ad))
#    print(f"Contents of a_d[{idx}]:", ad)


a_d = [steering_vector(theta, N, d, k,(x_opt)) for theta in theta_desired]  # Desired direction steering vectors

a_n = []#[steering_vector(theta, N, d, k,0) for theta in theta_interference]  # Interference direction steering vectors

# Define the covariance matrix (for simplicity, assume identity matrix as a placeholder)
R = np.eye(3)  # Covariance matrix of the received signal

# Define the constraints: desired directions and interference nulls
# Desired direction: a_d^H * w = 1 for each desired direction
# Interference directions: a_n^H * w = 0 for each interference direction
# Constraints matrix (stacked for MIMO and interference nulls)
A = np.vstack([np.array([ad.conj() for ad in a_d]), np.array([an.conj() for an in a_n])])  # Stack the desired and interference constraints
#A = np.array([ad.conj() for ad in a_d])
b = np.array([1] * len(a_d) + [0] * (len(a_n)))  # Right-hand side of constraints for MIMO and interference


import numpy as np
from scipy import linalg

# Ensure A, R, and b are numerical arrays
A = np.asarray(A, dtype=np.complex128)  # Convert to complex type
R = np.asarray(R, dtype=np.complex128)
b = np.asarray(b, dtype=np.complex128)
print(type(A), A.dtype)
print(type(R), R.dtype)
print(type(b), b.dtype)

# Solve the LCMV optimization problem using a linear system
# Minimize: w^H * R * w, subject to A * w = b
w = linalg.lstsq(A.T @ R @ A, A.T @ R @ b)[0]  # Solve using least squares

# Normalize the weights (optional)
w = w / np.linalg.norm(w)

# Plot the power pattern (beamforming radiation pattern)
theta_scan = np.linspace(-90, 90, 500)  # Scan angles
power_pattern = np.zeros_like(theta_scan)

for i, theta in enumerate(theta_scan):
    a = steering_vector(theta, N, d, k,(x_opt))  # Compute steering vector for each scan angle
    power_pattern[i] = np.abs(np.dot(w.conj(), a))**2  # Calculate the power

# Normalize the power pattern and convert to dB
power_pattern_db = 10 * np.log10(power_pattern / np.max(power_pattern))

# Plot the power pattern
plt.figure(figsize=(8, 6))
plt.plot(theta_scan, power_pattern_db)
plt.title('Beamforming Power Pattern (LCMV) for MIMO')
plt.xlabel('Scan Angle (Degrees)')
plt.ylabel('Power (dB)')
plt.grid(True)
plt.xlim([-90, 90])
plt.ylim([-50, 0])

# Mark the desired directions
for theta in theta_desired:
    plt.axvline(x=theta, color='r', linestyle='--', label=f'Desired Direction {theta}°')

plt.legend()
plt.show()


C:\Users\luso\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\cvxpy\expressions\expression.py:667: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 4 times so far.

  warnings.warn(msg, UserWarning)
C:\Users\luso\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\cvxpy\expressions\expression.py:667: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-mat

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 3 and the array at index 1 has size 0

In [41]:
xx = np.array([1+3e3j, 4+4j, 3])
print(xx.dtype)

complex128
